In [1]:
import pandas as pd
import re

def normalize_string(s):
    return re.sub(r'[^\w\s]', '', s).lower()

sites_df = pd.read_excel('health-care-capacity-dashboard-10-16-2024_updated.xlsx', sheet_name=None)

In [ ]:
hospital_df = sites_df['Hospital Information']
hospital_df['Facility type'] = 'Hospital'
hospital_df.rename(columns={'Short name': 'Facility name'}, inplace=True)

snf_df = sites_df['Nursing home information']
dist_df = sites_df['Nursing homes within 30 minutes']

In [4]:
hospital_df['norm_address'] = hospital_df['Address'].apply(normalize_string)
hospital_df['norm_name'] = hospital_df['Facility name'].apply(normalize_string)
hospital_df['norm_full_name'] = hospital_df['Hospital'].apply(normalize_string)

snf_df['norm_address'] = snf_df['Address'].apply(normalize_string)
snf_df['norm_name'] = snf_df['Facility name'].apply(normalize_string)

dist_df['norm_hosp'] = dist_df['Hospital'].apply(normalize_string)
dist_df['norm_snf'] = dist_df['Nursing homes within 30 minutes drive time'].apply(normalize_string)

In [6]:
snf_30_count = dist_df[['norm_hosp', 'Nursing homes within 30 minutes drive time']].groupby(['norm_hosp']).count()
hosp_30_count = dist_df[['norm_snf', 'norm_hosp']].groupby(['norm_snf']).count()
hosp_30_count.rename(columns={'norm_hosp': 'Hospitals within 30 minutes drive time'}, inplace=True)

In [10]:
mass_df = pd.merge(hospital_df, snf_30_count, how='left', left_on='norm_full_name', right_on='norm_hosp')

facility_info = pd.concat([mass_df, snf_df], ignore_index=True)
facility_info = pd.merge(facility_info, hosp_30_count, how='left', left_on='norm_name', right_on='norm_snf')

facility_info['Hospitals within 30 minutes drive time'] = facility_info['Hospitals within 30 minutes drive time'].combine_first(facility_info['Number of hospitals within 30 minute drive time'])
facility_info['norm_name'] = facility_info['norm_full_name'].combine_first(facility_info['norm_name'])

facility_info.drop(columns=['Number of hospitals within 30 minute drive time', 'norm_full_name'], inplace=True)

In [16]:
facility_info

,IDNUMBER,Hospital,Facility name,Address,Town,ZIP code,CHIA region,Type of hospital,Hospital system,Telephone,Facility type,norm_address,norm_name,Nursing homes within 30 minutes drive time,Hospitals within 30 minutes drive time
0,2014,North Shore Medical Center - Salem Hospital,Salem Hospital,81 Highland Avenue,Salem,1970,Northeastern Massachusetts,Community-High Public Payer Hospital,Mass General Brigham,(978) 741-1200,Hospital,81 highland avenue,north shore medical center salem hospital,28.0,3.0
1,2059,New England Baptist Hospital,New England Baptist Hospital,125 Parker Hill Avenue,Boston,2120,Metro Boston,Specialty Hospital,Beth Israel Lahey Health,(617) 754-5000,Hospital,125 parker hill avenue,new england baptist hospital,48.0,18.0
2,2107,South Shore Hospital,South Shore Hospital,55 Fogg Road,Weymouth,2190,Metro South,Community Hospital,South Shore Health System,(781) 624-8000,Hospital,55 fogg road,south shore hospital,53.0,11.0
3,2058,Melrose-Wakefield Hospital,Melrose-Wakefield Hospital,585 Lebanon Street,Melrose,2176,Metro Boston,Community-High Public Payer Hospital,Tufts Medicine,(781) 979-3000,Hospital,585 lebanon street,melrosewakefield hospital,56.0,16.0
4,2022,Steward Morton Hospital,Morton Hospital,88 Washington Street,Taunton,2780,Metro South,Community-High Public Payer Hospital,Steward Health Care,(508) 828-7000,Hospital,88 washington street,steward morton hospital,31.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,NaN,NaN,Cedar View Rehabilitation and Healthcare Center,480 Jackson Street,Methuen,1844,Northeastern Massachusetts,NaN,NaN,NaN,Nursing home,480 jackson street,cedar view rehabilitation and healthcare center,NaN,9.0
423,NaN,NaN,Life Care Center of the South Shore,309 Driftway,Scituate,2066,Metro South,NaN,NaN,NaN,Nursing home,309 driftway,life care center of the south shore,NaN,1.0
424,NaN,NaN,German Center for Extended Care,2222 Centre Street,Boston,2132,Metro Boston,NaN,NaN,NaN,Nursing home,2222 centre street,german center for extended care,NaN,17.0
425,NaN,NaN,Vantage at South Hadley,573 Granby Road,South Hadley,1075,Western Massachusetts,NaN,NaN,NaN,Nursing home,573 granby road,vantage at south hadley,NaN,6.0
